In [2]:
import json
import glob, os
from itertools import combinations

USER_STUDIES_DIR = "/home/shared/vlm_rationales_eval/user_studies_data/prolific_batches/batch_interaction_data"
print(os.listdir(USER_STUDIES_DIR))

['showbothmetrics_llava1.5_with_image_q20_i10_s0', 'llava1.5_with_image_q20_i10_s0', 'prodmetric_llava1.5_with_image_q20_i10_s0', 'vf_numeric_llava1.5_with_image_q20_i10_s0', 'contr_numeric_llava1.5_with_image_q20_i10_s0', 'avg_vf_contr_numeric_llava1.5_with_image_q20_i10_s0', 'showbothmetrics_descriptive_llava1.5_with_image_q20_i10_s0']


In [3]:
import json
import glob, os
from itertools import combinations

USER_STUDIES_DIR = "/home/shared/vlm_rationales_eval/user_studies_data/"
print(os.listdir(USER_STUDIES_DIR))
settings = [
    'showbothmetrics_llava1.5_with_image_q20_i10_s0',
    'prodmetric_llava1.5_with_image_q20_i10_s0',
    'vf_numeric_llava1.5_with_image_q20_i10_s0',
    'contr_numeric_llava1.5_with_image_q20_i10_s0', 
    'avg_vf_contr_numeric_llava1.5_with_image_q20_i10_s0'
]

# Dictionary to hold the set of user_ids for each setting.
settings_user_ids = {}

for setting in settings:
    print(f"Processing setting: {setting}")
    pattern = os.path.join(USER_STUDIES_DIR, "prolific_batches", "batch_interaction_data", setting, "*.json")
    files = glob.glob(pattern)
    user_ids = set()
    
    for file in files:
        with open(file) as f:
            batch_data = json.load(f)
            # Add all user_ids from the current JSON file.
            user_ids.update(batch_data.keys())
    
    settings_user_ids[setting] = user_ids
    print(f"{len(user_ids)} participants in {setting}")

# Compare overlaps between each pair of settings.
print("\nOverlap between settings:")
for s1, s2 in combinations(settings, 2):
    overlap = settings_user_ids[s1].intersection(settings_user_ids[s2])
    print(f"Overlap between {s1} and {s2}: {len(overlap)} participants")
    
# Optionally, check for participants common to all three settings.
triple_overlap = settings_user_ids[settings[0]].intersection(settings_user_ids[settings[1]], settings_user_ids[settings[2]])
print(f"\nParticipants present in all three settings: {len(triple_overlap)}")


['prolific_batches']
Processing setting: showbothmetrics_llava1.5_with_image_q20_i10_s0
30 participants in showbothmetrics_llava1.5_with_image_q20_i10_s0
Processing setting: prodmetric_llava1.5_with_image_q20_i10_s0
31 participants in prodmetric_llava1.5_with_image_q20_i10_s0
Processing setting: vf_numeric_llava1.5_with_image_q20_i10_s0
28 participants in vf_numeric_llava1.5_with_image_q20_i10_s0
Processing setting: contr_numeric_llava1.5_with_image_q20_i10_s0
30 participants in contr_numeric_llava1.5_with_image_q20_i10_s0
Processing setting: avg_vf_contr_numeric_llava1.5_with_image_q20_i10_s0
31 participants in avg_vf_contr_numeric_llava1.5_with_image_q20_i10_s0

Overlap between settings:
Overlap between showbothmetrics_llava1.5_with_image_q20_i10_s0 and prodmetric_llava1.5_with_image_q20_i10_s0: 0 participants
Overlap between showbothmetrics_llava1.5_with_image_q20_i10_s0 and vf_numeric_llava1.5_with_image_q20_i10_s0: 1 participants
Overlap between showbothmetrics_llava1.5_with_image

In [21]:
import json
import numpy as np
import glob, os

USER_STUDIES_DIR = "/home/shared/vlm_rationales_eval/user_studies_data/"

# setting = 'showbothmetrics_llava1.5_with_image_q20_i10_s0'
# setting = 'showbothmetrics_descriptive_llava1.5_with_image_q20_i10_s0'
# setting = 'prodmetric_llava1.5_with_image_q20_i10_s0'
# setting = 'vf_numeric_llava1.5_with_image_q20_i10_s0'
# setting = 'contr_numeric_llava1.5_with_image_q20_i10_s0'
setting = 'avg_vf_contr_numeric_llava1.5_with_image_q20_i10_s0'
print(f"Setting: {setting}")
files = glob.glob(f'{USER_STUDIES_DIR}/prolific_batches/batch_interaction_data/{setting}/*.json')
data = {}
for file in files:
    with open(file) as f:
        batch_data = json.load(f)
        for user_id, session in batch_data.items():
            if user_id in data:
                data[user_id].append(session)  # Append to list instead of overwriting
            else:
                data[user_id] = [session]  # Initialize as list

user_ids = list(data.keys())
for user_id in user_ids:
    interactions = sum([x['interactions'] for x in data[user_id]], [])
    user_preds = np.array([interaction['user_selections']['answeronly'] for interaction in interactions])
    unsure_rate = np.mean(user_preds == 2)
    if unsure_rate < 0.2:
        del data[user_id]

print(f"{sum(len(sessions) for sessions in data.values())} total sessions loaded from {len(files)} files")

all_instances = sum([session['interactions'] for sessions in data.values() for session in sessions], [])

Setting: avg_vf_contr_numeric_llava1.5_with_image_q20_i10_s0
31 total sessions loaded from 10 files


In [22]:
# Evaluate the interactions
from collections import Counter
import math

def compute_margin(p, n):
    return 1.96*math.sqrt(p*(1-p)/n) if n > 0 else 0

def compute_proportion_ci(p, n):
    margin = compute_margin(p, n)
    return (p - margin, p + margin)

def compute_mean_ci(values):
    n = len(values)
    mean_val = np.mean(values)
    std_val = np.std(values, ddof=1)
    se = std_val/np.sqrt(n) if n > 0 else 0
    return (mean_val - 1.96*se, mean_val + 1.96*se)

def evaluate_answers(stage, instances):
    ground_truths = np.array([1-x['question']['prediction_is_correct'] for x in instances])       # 0 means AI is correct, 1 means AI is incorrect
    preds = np.array([x['user_selections'][stage] for x in instances])
    true_positives = np.sum(np.logical_and(preds == 0, ground_truths == 0))
    false_positives = np.sum(np.logical_and(preds == 0, ground_truths == 1))
    true_negatives = np.sum(np.logical_and(preds == 1, ground_truths == 1))
    false_negatives = np.sum(np.logical_and(preds == 1, ground_truths == 0))

    unsure_rate = np.mean(preds == 2)
    accuracy = (true_positives + true_negatives) / (true_positives + false_positives + true_negatives + false_negatives)
    total_accuracy = (true_positives + true_negatives) / len(ground_truths)
    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    false_positive_rate = false_positives / (false_positives + true_negatives) if false_positives + true_negatives > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

    # Utility: 0 if user is unsure, 1 if user correctly predicts AI correctness, -1 if user incorrectly predicts AI correctness
    individual_utilities = np.array([0 if preds[i] == 2 else 1-2*np.abs(preds[i] - ground_truths[i]) for i in range(len(preds))])
    utility = np.mean(individual_utilities)
    
    # Compute 95% CIs.
    total_accuracy_ci = compute_proportion_ci(total_accuracy, len(ground_truths))
    unsure_rate_ci    = compute_proportion_ci(unsure_rate, len(preds))
    
    non_unsure = preds != 2
    non_unsure_n = np.sum(non_unsure)
    accuracy_ci = compute_proportion_ci(accuracy, non_unsure_n) if non_unsure_n > 0 else (0, 0)
    
    precision_n = true_positives + false_positives
    precision_ci = compute_proportion_ci(precision, precision_n) if precision_n > 0 else (0, 0)
    
    recall_n = true_positives + false_negatives
    recall_ci = compute_proportion_ci(recall, recall_n) if recall_n > 0 else (0, 0)
    
    fpr_n = false_positives + true_negatives
    fpr_ci = compute_proportion_ci(false_positive_rate, fpr_n) if fpr_n > 0 else (0, 0)
    
    utility_ci = compute_mean_ci(individual_utilities)

    return {
        'total_accuracy': total_accuracy,
        'total_accuracy_ci': total_accuracy_ci,
        'accuracy': accuracy,
        'accuracy_ci': accuracy_ci,
        'precision': precision,
        'precision_ci': precision_ci,
        'recall': recall,
        'recall_ci': recall_ci,
        'false_positive_rate': false_positive_rate,
        'fpr_ci': fpr_ci,
        'f1': f1,
        'unsure_rate': unsure_rate,
        'unsure_rate_ci': unsure_rate_ci,
        'utility': utility,
        'utility_ci': utility_ci,
        'preds': preds,
        'ground_truths': ground_truths,
        'individual_utilities': individual_utilities,
    }

answeronly_results = evaluate_answers('answeronly', all_instances)
withexplanation_results = evaluate_answers('withexplanation', all_instances)
withexplanationquality_results = evaluate_answers('withexplanationquality', all_instances)

# print("Stage                   \tUnsure Rate\tTotalAcc\tNotUnsureAcc\tPrecision\tRecall\t\tF1\t\tFPR\t\tUtility")
# print("-"*140)
# for stage, results in zip(
#     ['Answer Only', 'With Explanation', 'With Explanation + Quality'], 
#     [answeronly_results, withexplanation_results, withexplanationquality_results]
# ):
#     print(f"{stage:<25}\t{results['unsure_rate']:.1%}\t\t{results['total_accuracy']:.1%}\t\t{results['accuracy']:.1%}\t\t{results['precision']:.3f}\t\t{results['recall']:.3f}\t\t{results['f1']:.3f}\t\t{results['false_positive_rate']:.3f}\t\t{results['utility']:.3f}")
    
# print("Stage                   \tUnsure Rate\tTotalAcc\tNotUnsureAcc\tPrecision\tRecall\t\tF1\tFPR\t\tUtility")
print("Stage                   \tUnsure Rate\tTotalAcc\tNotUnsureAcc\tUtility")
# print("-"*150) 
print("-"*100)
for stage_name, results in zip(
    ['Answer Only', 'With Explanation', 'With Explanation + Quality'], 
    [answeronly_results, withexplanation_results, withexplanationquality_results]
):
    # Calculate margins as half the width of each CI.
    unsure_margin    = (results['unsure_rate_ci'][1] - results['unsure_rate_ci'][0]) / 2
    total_acc_margin = (results['total_accuracy_ci'][1] - results['total_accuracy_ci'][0]) / 2
    acc_margin       = (results['accuracy_ci'][1] - results['accuracy_ci'][0]) / 2
    prec_margin      = (results['precision_ci'][1] - results['precision_ci'][0]) / 2 if results['precision_ci'] != (0, 0) else 0
    recall_margin    = (results['recall_ci'][1] - results['recall_ci'][0]) / 2 if results['recall_ci'] != (0, 0) else 0
    fpr_margin       = (results['fpr_ci'][1] - results['fpr_ci'][0]) / 2 if results['fpr_ci'] != (0, 0) else 0
    utility_margin   = (results['utility_ci'][1] - results['utility_ci'][0]) / 2

    # For proportion metrics, we print as percentages; f1 is printed as a decimal (CI not computed); utility remains as a number.
    print(f"{stage_name:<25}\t"
          f"{results['unsure_rate']:.1%} ± {unsure_margin:.1%}\t"
          f"{results['total_accuracy']:.1%} ± {total_acc_margin:.1%}\t"
          f"{results['accuracy']:.1%} ± {acc_margin:.1%}\t"
        #   f"{results['precision']:.1%} ± {prec_margin:.1%}\t"
        #   f"{results['recall']:.1%} ± {recall_margin:.1%}\t"
        #   f"{results['f1']:.3f}\t"
        #   f"{results['false_positive_rate']:.1%} ± {fpr_margin:.1%}\t"
          f"{results['utility']:.3f} ± {utility_margin:.3f}")
    
    # print("copiable results:", end=' ')
    # print(f"{results['unsure_rate']:.1%} ± {unsure_margin:.1%}, ", end='')
    # print(f"{results['total_accuracy']:.1%} ± {total_acc_margin:.1%}, ", end='')
    # print(f"{results['accuracy']:.1%} ± {acc_margin:.1%}, ", end='')
    # print(f"{results['utility']:.3f} ± {utility_margin:.3f}")

print("\nSetting:", setting)
print("Stage\t\tUnsure Rate\tUtility over previous stage")
answeronly_individual_utilities = answeronly_results['individual_utilities']
explanation_individual_utilities = withexplanation_results['individual_utilities']
explanationquality_individual_utilities = withexplanationquality_results['individual_utilities']
assert len(explanation_individual_utilities) == len(explanationquality_individual_utilities)
assert len(answeronly_individual_utilities) == len(explanation_individual_utilities)
answeronly_preds = answeronly_results['preds']
explanation_preds = withexplanation_results['preds']
explanationquality_preds = withexplanationquality_results['preds']
assert len(explanation_preds) == len(explanationquality_preds)
assert len(answeronly_preds) == len(explanation_preds)

answeronly_unsurepreds = len([apred for apred in answeronly_preds if apred == 2])
explanation_nonunsurepreds = len([apred for apred, epred in zip(answeronly_preds, explanation_preds) if apred == 2 & epred != 2])
explanation_unsuredecrease = (answeronly_unsurepreds - explanation_nonunsurepreds) / answeronly_unsurepreds

explanationquality_nonunsurepreds = len([apred for apred, epred in zip(answeronly_preds, explanationquality_preds) if apred == 2 & epred != 2])
explanationquality_unsuredecrease = (answeronly_unsurepreds - explanationquality_nonunsurepreds) / answeronly_unsurepreds

explanation_utility_gains = [explanation - answeronly for answeronly, explanation in zip(answeronly_individual_utilities, explanation_individual_utilities)]
explanation_utility_gains_ci = compute_mean_ci(explanation_utility_gains)
explanation_utility_gains_ci_margin = (explanation_utility_gains_ci[1] - explanation_utility_gains_ci[0]) / 2
print("Explanation\t", f"{withexplanation_results['unsure_rate']:.1%}    \t", f"{np.mean(explanation_utility_gains):.3f} ± {explanation_utility_gains_ci_margin:.3f}")



quality_utility_gains = [quality - explanation for explanation, quality in zip(explanation_individual_utilities, explanationquality_individual_utilities)]
quality_utility_gains_ci = compute_mean_ci(quality_utility_gains)
quality_utility_gains_ci_margin = (quality_utility_gains_ci[1] - quality_utility_gains_ci[0]) / 2
print(f"Quality\t\t", f"{withexplanationquality_results['unsure_rate']:.1%}    \t", f"{np.mean(quality_utility_gains):.3f} ± {quality_utility_gains_ci_margin:.3f}")
    
def num(stage2_utility, stage3_utility):
    return len([1 for u1, u2 in zip(explanation_individual_utilities, explanationquality_individual_utilities) if u1 == stage2_utility and u2 == stage3_utility])

print(f"\nSetting: {setting.split('_ll')[0]}")
print("\t\tIncorrect\tUnsure   \tCorrect")
for i, s in zip([-1, 0, 1], ['Incorrect', 'Unsure   ', 'Correct  ']):
    print(s, "\t", '\t\t'.join([str(num(j, i)) for j in [-1, 0, 1]]))

# M1: P(S3 != U | S2 = U)
m1 = (num(0, 1) + num(0, -1)) / (num(0, 1) + num(0, -1) + num(0, 0))

# M2: P(S3 = C | S2 = U, S3 != U)
m2 = num(0, 1) / (num(0, 1) + num(0, -1))

# M3: P(S3 != I | S2 = I)
m3 = (num(-1, 0) + num(-1, 1)) / (num(-1, 0) + num(-1, 1) + num(-1, -1))

# M4: P(S3 != C | S2 = C)
m4 = (num(1, 0) + num(1, -1)) / (num(1, 0) + num(1, -1) + num(1, 1))
print("M1\tM2\tM3\tM4")
print(f"{m1:.2%}\t{m2:.2%}\t{m3:.2%}\t{m4:.2%}")


Stage                   	Unsure Rate	TotalAcc	NotUnsureAcc	Utility
----------------------------------------------------------------------------------------------------
Answer Only              	85.2% ± 4.0%	8.1% ± 3.0%	54.3% ± 14.4%	0.013 ± 0.043
With Explanation         	34.2% ± 5.3%	41.6% ± 5.5%	63.2% ± 6.6%	0.174 ± 0.088
With Explanation + Quality	6.1% ± 2.7%	59.4% ± 5.5%	63.2% ± 5.5%	0.248 ± 0.104

Setting: avg_vf_contr_numeric_llava1.5_with_image_q20_i10_s0
Stage		Unsure Rate	Utility over previous stage
Explanation	 34.2%    	 0.161 ± 0.088
Quality		 6.1%    	 0.074 ± 0.070

Setting: avg_vf_contr_numeric
		Incorrect	Unsure   	Correct
Incorrect 	 66		40		1
Unsure    	 6		8		5
Correct   	 3		58		123
M1	M2	M3	M4
92.45%	59.18%	12.00%	4.65%


In [ ]:
import json
import numpy as np
import glob, os

USER_STUDIES_DIR = "/home/shared/vlm_rationales_eval/user_studies_data/"

# setting = 'showbothmetrics_llava1.5_with_image_q20_i10_s0'
# setting = 'prodmetric_llava1.5_with_image_q20_i10_s0'
def load_data(setting):
    #print(f"Setting: {setting}")
    files = glob.glob(f'{USER_STUDIES_DIR}/prolific_batches/batch_interaction_data/{setting}/*.json')
    data = {}
    for file in files:
        with open(file) as f:
            batch_data = json.load(f)
            for user_id, session in batch_data.items():
                if user_id in data:
                    print(user_id, setting, session['interactions'][0]['url_data']['uid'], data[user_id][0]['interactions'][0]['url_data']['uid'])
                    data[user_id].append(session)  # Append to list instead of overwriting
                else:
                    data[user_id] = [session]  # Initialize as list

    #print(f"{sum(len(sessions) for sessions in data.values())} total sessions loaded from {len(files)} files")
    return data

settings = [
    'showbothmetrics', 
    'vf_numeric', 
    'contr_numeric', 
    'avg_vf_contr_numeric', 
]
data = {setting: load_data(f"{setting}_llava1.5_with_image_q20_i10_s0") for setting in settings}

from collections import defaultdict
counter = defaultdict(lambda: defaultdict(int))
for setting, user_data in data.items():
    all_instances = sum([session['interactions'] for sessions in user_data.values() for session in sessions], [])
    for instance in all_instances:
        counter[instance['question']['question_id']][setting] += 1


In [ ]:
qids = list(counter.keys())
qids.sort()
print("QID\t" + '\t'.join(settings))
for qid in qids:
    print(qid + "\t" + '\t\t'.join(f'{counter[qid][setting]}' for setting in settings))
for setting in settings:
    counts = [counter[qid][setting] for qid in qids]
    print(setting)
    print("\tQuestions with < 3 annotations:", sum(1 for count in counts if count < 3))
    print("\tQuestions with > 3 annotations:", sum(1 for count in counts if count > 3))

In [ ]:
print("Setting             \tQueue #\t Study ID                 \tProlific User ID          \tInitial (answer-only) Unsure Rate")
print("-"*120)

for setting in settings:
    #print(f"Setting: {setting}")
    for user_id in data[setting]:
        interactions = sum([x['interactions'] for x in data[setting][user_id]], [])
        user_preds = np.array([interaction['user_selections']['answeronly'] for interaction in interactions])
        unsure_rate = np.mean(user_preds == 2)
        # print(f"\t{user_id}: {unsure_rate:.0%} unsure rate")
        if unsure_rate < 0.2:
            # print("\t\t", user_id, setting, data[setting][user_id][0]['interactions'][0]['url_data']['uid'])
            #print(user_id, "\t", setting, "    \t", data[setting][user_id][0]['interactions'][0]['url_data']['study_id'])
            print(setting, "     \t",data[setting][user_id][0]['interactions'][0]['url_data']['uid'].split('/')[-1], "\t", data[setting][user_id][0]['interactions'][0]['url_data']['study_id'], "\t", user_id, "\t", unsure_rate)
